Included installs

In [1]:
# !pip install pandas

In [2]:
# !pip install influxdb

In [3]:
import pandas as pd, numpy as np
from influxdb import DataFrameClient
import json

Set params

In [8]:
purge=False
write_out=True
full_overwrite=False

Establish DB connection

In [5]:
user = 'admin'
password = open('auth/influxa.txt','r').read()
host='influxdb'
port=8086
dbname='base'
protocol = 'line' #'json'
client = DataFrameClient(host, port, user, password, dbname)

In [6]:
if purge:
    client.drop_database(dbname)
    client.drop_retention_policy(dbname)
    client.create_database(dbname)
    client.create_retention_policy(dbname, '3300d', 1, default=True)

In [9]:
htmlipath='../html/'

In [10]:
import requests
grafana = "http://grafana:3000/"
headers = {
    'Authorization': 'Bearer '+open('auth/grafana.txt','r').read(),
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

In [11]:
response = requests.get(grafana+'api/folders', headers=headers)
folders=json.loads(response.content)

In [12]:
import time
import datetime
import pytz
utc=pytz.UTC

In [13]:
def push2influx(df,measurement,field_columns,tag_columns,shift=False,dbclient=client,wo=write_out,fo=full_overwrite,daily=True):
    if wo:
        df=df.sort_index()
        df.index=df.index.tz_localize('GMT')
        if shift:
            df.index+=pd.to_timedelta('12h')
        if fo: 
            print('Purging',measurement,'...')
            dbclient.query('DROP MEASUREMENT '+measurement)
        else:
            latest=dbclient.query('SELECT * FROM '+measurement+' GROUP BY "1d" ORDER BY DESC LIMIT 1')
            if latest:
                lat=latest[list(latest.keys())[0]].index[0]
                if daily: lat+=pd.to_timedelta('1d')
                df=df[lat:]
                print('Slicing',measurement,'from',lat,'...')
            else:
                print('No data in db for',measurement,'...')
        time.sleep(3)
        print('Writing to',measurement,'...')
        bsize=5000
        bwait=2
        print(len(df),'data points will be written in',len(df)/bsize,'batches.')
        print('Expected query running time is:',int((len(df)/bsize)*bwait*1.1)+3,'seconds.')
        for i in range(int(len(df)/bsize)+1):
            r=range(i*bsize,min(len(df),(i+1)*bsize))
            dc=df.iloc[r]
            print('Writing batch',i+1,'...')
            dbclient.write_points(dc, measurement, protocol=protocol,
                                field_columns=field_columns,
                                tag_columns=[])
            time.sleep(bwait)
        time.sleep(3)
        print('Done!')
    else:
        print('Write-out not enabled. Skipping...')

Fetch stock names

In [87]:
import requests
url = "https://api.iextrading.com/1.0/ref-data/symbols"
result = requests.get(url).json()
tickers=pd.DataFrame(result)

In [88]:
# !pip install pandas_ta

In [89]:
import pandas_ta as ta

In [90]:
import warnings
warnings.filterwarnings("ignore")

Daily

In [91]:
df=pd.read_csv('../data/all_stocks_5yr.csv')
df['date']=df['date'].astype(np.datetime64)
pd.to_datetime('now')-df['date'].min()

Timedelta('3189 days 10:57:14.440091')

In [92]:
df=df.set_index('date')
df.head()

,open,high,low,close,volume,Name
date,,,,,,
2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [93]:
df_stack=df.reset_index().set_index(['Name','date'])

In [125]:
import pandas_datareader as pdr
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [134]:
dfs=[]
# for i in df_stack.index.get_level_values(0).unique():
for i in ['AAL', 'AAPL']:
    try:
        dfa=df_stack.loc[i]
        dfa.ta.macd(close='close', fast=12, slow=26, signal=9, append=True)
        dfa.ta.bbands(20,append=True)
        dfa['ticker']=i
        
        dfa["Diff"] = dfa.close.diff()
        dfa["SMA_2"] = dfa.close.rolling(2).mean()
        dfa["Force_Index"] = dfa.close * dfa.volume
        dfa["y"] = dfa["Diff"].apply(lambda x: 1 if x > 0 else 0).shift(-1)
        dfb = dfa[['SMA_2','Force_Index','y']].dropna()

        X = dfb.drop(["y"], axis=1).values
        y = dfb["y"].values
        X_train, X_test, y_train, y_test = train_test_split(
           X,
           y,
           test_size=0.2,
           shuffle=False,
        )
        clf = make_pipeline(StandardScaler(), MLPClassifier(random_state=0, shuffle=False))
        clf.fit(
           X_train,
           y_train,
        )
        y_pred = clf.predict(X_test)
        dfa['mlp']=y_pred[-1]
        
        dfs.append(dfa)
    except:
        pass

In [137]:
dfs=pd.concat(dfs)

In [138]:
df=dfs.join(tickers.set_index('symbol')['name'],on='ticker')
df['name']=df['name'].str.replace(' INC','').str.title()

df['diff']=df['close']-df['open']
direction=[]
for i in df['diff'].values:
    if i<0:
        direction.append('minus')
    else:
        direction.append('plus')
df['direction']=direction
df['diff']=abs(df['diff'])

direction=[]
for i in df['MACDh_12_26_9'].values:
    if i<0:
        direction.append('minus')
    else:
        direction.append('plus')
df['MACDh_direction']=direction

df['bollingerm']=df['close']-df['BBM_20_2.0']
df['bollingeru']=df['close']-df['BBU_20_2.0']
df['bollingerl']=df['close']-df['BBL_20_2.0']
direction=[]
for i in df[['bollingerm','bollingeru','bollingerl']].values:
    if i[0]>0.1:
        if i[1]>0:
            direction.append(-1)
        else:
            direction.append(-0.5)
    elif i[0]<-0.1:
        if i[2]>0:
            direction.append(0.5)
        else:
            direction.append(1)
    else:
        direction.append(0)
df['bollinger']=direction

df['overall']=df['bollinger']+(df['mlp']-0.5)*2+df['MACDh_12_26_9']

In [151]:
df1=df[df['ticker'].isin(['AAPL','AAL'])]

Push to Influx

In [152]:
tag_columns=['name','direction','ticker','MACDh_direction']
field_columns=[i for i in df.columns if i not in tag_columns]
measurement='daily'
push2influx(df1,measurement,field_columns,tag_columns,fo=True)

Purging daily ...
Writing to daily ...
2518 data points will be written in 0.5036 batches.
Expected query running time is: 4 seconds.
Writing batch 1 ...
Done!


Deep learning predictions

In [141]:
# !pip install tensorflow

In [142]:
# !pip install scikit-learn keras pandas_datareader

LSTM

In [116]:
# https://10mohi6.medium.com/super-easy-python-stock-price-forecast-using-keras-lstm-deep-learning-bd972bcca70
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Dense, LSTM
import pandas_datareader as pdr
from sklearn.metrics import accuracy_score

dfa=df_stack.loc[i]
dfa["Diff"] = dfa.close.diff()
dfa["SMA_2"] = dfa.close.rolling(2).mean()
dfa["Force_Index"] = dfa.close * dfa.volume
dfa["y"] = dfa["Diff"].apply(lambda x: 1 if x > 0 else 0).shift(-1)
dfa = dfa[['SMA_2','Force_Index','y']].dropna()
X = StandardScaler().fit_transform(dfa.drop(["y"], axis=1))
y = dfa["y"].values
X_train, X_test, y_train, y_test = train_test_split(
   X,
   y,
   test_size=0.2,
   shuffle=False,
)
model = Sequential()
model.add(LSTM(2, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model.fit(X_train[:, :, np.newaxis], y_train, epochs=20)
y_pred = model.predict(X_test[:, :, np.newaxis])
print(accuracy_score(y_test, y_pred > 0.5))

Epoch 1/20
32/32 [==============================] - 3s 12ms/step - loss: 0.6959 - acc: 0.5154
Epoch 2/20
32/32 [==============================] - 0s 13ms/step - loss: 0.6953 - acc: 0.5134
Epoch 3/20
32/32 [==============================] - 0s 13ms/step - loss: 0.6948 - acc: 0.5134
Epoch 4/20
32/32 [==============================] - 0s 10ms/step - loss: 0.6946 - acc: 0.5154
Epoch 5/20
32/32 [==============================] - 0s 13ms/step - loss: 0.6942 - acc: 0.5045
Epoch 6/20
32/32 [==============================] - 0s 12ms/step - loss: 0.6940 - acc: 0.5075
Epoch 7/20
32/32 [==============================] - 0s 13ms/step - loss: 0.6938 - acc: 0.5085
Epoch 8/20
32/32 [==============================] - 0s 12ms/step - loss: 0.6937 - acc: 0.5134
Epoch 9/20
32/32 [==============================] - 0s 10ms/step - loss: 0.6936 - acc: 0.5095
Epoch 10/20
32/32 [==============================] - 0s 10ms/step - loss: 0.6935 - acc: 0.5095
Epoch 11/20
32/32 [==============================] - 0s 13m

MLP

In [131]:
# https://10mohi6.medium.com/super-easy-python-stock-price-forecasting-using-multilayer-perceptron-machine-learning-4f1d1ef9650
import pandas_datareader as pdr
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

dfa=df_stack.loc[i]
dfa["Diff"] = dfa.close.diff()
dfa["SMA_2"] = dfa.close.rolling(2).mean()
dfa["Force_Index"] = dfa.close * dfa.volume
dfa["y"] = dfa["Diff"].apply(lambda x: 1 if x > 0 else 0).shift(-1)
dfa = dfa[['SMA_2','Force_Index','y']].dropna()

X = dfa.drop(["y"], axis=1).values
y = dfa["y"].values
X_train, X_test, y_train, y_test = train_test_split(
   X,
   y,
   test_size=0.2,
   shuffle=False,
)
clf = make_pipeline(StandardScaler(), MLPClassifier(random_state=0, shuffle=False))
clf.fit(
   X_train,
   y_train,
)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.5357142857142857


GRU

In [123]:
# https://10mohi6.medium.com/super-easy-python-stock-price-forecast-using-keras-gru-deep-learning-a85dddce54db
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Dense, GRU
import pandas_datareader as pdr
from sklearn.metrics import accuracy_score

dfa=df_stack.loc[i]
dfa["Diff"] = dfa.close.diff()
dfa["SMA_2"] = dfa.close.rolling(2).mean()
dfa["Force_Index"] = dfa.close * dfa.volume
dfa["y"] = dfa["Diff"].apply(lambda x: 1 if x > 0 else 0).shift(-1)
dfa = dfa[['SMA_2','Force_Index','y']].dropna()

X = StandardScaler().fit_transform(dfa.drop(["y"], axis=1))
y = dfa["y"].values
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    shuffle=False,
)
model = Sequential()
model.add(GRU(2, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model.fit(X_train[:, :, np.newaxis], y_train, epochs=20)
y_pred = model.predict(X_test[:, :, np.newaxis])
print(accuracy_score(y_test, y_pred > 0.5))

Epoch 1/20
32/32 [==============================] - 4s 13ms/step - loss: 0.6960 - acc: 0.5194
Epoch 2/20
32/32 [==============================] - 0s 13ms/step - loss: 0.6955 - acc: 0.5294
Epoch 3/20
32/32 [==============================] - 0s 13ms/step - loss: 0.6948 - acc: 0.5264
Epoch 4/20
32/32 [==============================] - 0s 12ms/step - loss: 0.6945 - acc: 0.5284
Epoch 5/20
32/32 [==============================] - 0s 13ms/step - loss: 0.6941 - acc: 0.5254
Epoch 6/20
32/32 [==============================] - 0s 13ms/step - loss: 0.6938 - acc: 0.5264
Epoch 7/20
32/32 [==============================] - 0s 10ms/step - loss: 0.6936 - acc: 0.5234
Epoch 8/20
32/32 [==============================] - 0s 10ms/step - loss: 0.6935 - acc: 0.5254
Epoch 9/20
32/32 [==============================] - 0s 10ms/step - loss: 0.6933 - acc: 0.5234
Epoch 10/20
32/32 [==============================] - 0s 13ms/step - loss: 0.6931 - acc: 0.5264
Epoch 11/20
32/32 [==============================] - 0s 13m

Resample 7d

In [29]:
dfs=[]
# for i in df_stack.index.get_level_values(0).unique():
for i in ['AAL', 'AAPL']:
    try:
        dfa=df_stack.loc[i][['open','close','high','low','volume']]
        dfa['open']=dfa.interpolate().resample('7d').first()['open']
        dfa['close']=dfa.interpolate().resample('7d').last()['close']
        dfa['high']=dfa.interpolate().resample('7d').max()['high']
        dfa['low']=dfa.interpolate().resample('7d').min()['low']
        dfa['volume']=dfa.interpolate().resample('7d').sum()['volume']
        dfa=dfa.dropna()
        dfa.ta.macd(close='close', fast=12, slow=26, signal=9, append=True)
        dfa.ta.bbands(20,append=True)
        dfa['ticker']=i
        dfs.append(dfa)
    except:
        pass

In [30]:
dfs=pd.concat(dfs)

In [31]:
df=dfs.join(tickers.set_index('symbol')['name'],on='ticker')
df['name']=df['name'].str.replace(' INC','').str.title()

df['diff']=df['close']-df['open']
direction=[]
for i in df['diff'].values:
    if i<0:
        direction.append('minus')
    else:
        direction.append('plus')
df['direction']=direction
df['diff']=abs(df['diff'])

direction=[]
for i in df['MACDh_12_26_9'].values:
    if i<0:
        direction.append('minus')
    else:
        direction.append('plus')
df['MACDh_direction']=direction

df['bollingerm']=df['close']-df['BBM_20_2.0']
df['bollingeru']=df['close']-df['BBU_20_2.0']
df['bollingerl']=df['close']-df['BBL_20_2.0']
direction=[]
for i in df[['bollingerm','bollingeru','bollingerl']].values:
    if i[0]>0.1:
        if i[1]>0:
            direction.append(-1)
        else:
            direction.append(-0.5)
    elif i[0]<-0.1:
        if i[2]>0:
            direction.append(0.5)
        else:
            direction.append(1)
    else:
        direction.append(0)
df['bollinger']=direction

In [32]:
df1=df[df['ticker'].isin(['AAPL','AAL'])]

In [33]:
tag_columns=['name','direction','ticker','MACDh_direction']
field_columns=[i for i in df.columns if i not in tag_columns]
measurement='weekly'
push2influx(df1,measurement,field_columns,tag_columns,fo=True)

Purging weekly ...
Writing to weekly ...
504 data points will be written in 0.1008 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Resample 1 month

In [36]:
dfs=[]
# for i in df_stack.index.get_level_values(0).unique():
for i in ['AAL', 'AAPL']:
    try:
        dfa=df_stack.loc[i][['open','close','high','low','volume']]
        dfa['open']=dfa.interpolate().resample('1M').first()['open']
        dfa['close']=dfa.interpolate().resample('1M').last()['close']
        dfa['high']=dfa.interpolate().resample('1M').max()['high']
        dfa['low']=dfa.interpolate().resample('1M').min()['low']
        dfa['volume']=dfa.interpolate().resample('1M').sum()['volume']
        dfa=dfa.dropna()
        dfa.ta.macd(close='close', fast=12, slow=26, signal=9, append=True)
        dfa.ta.bbands(20,append=True)
        dfa['ticker']=i
        dfs.append(dfa)
    except:
        pass

In [37]:
dfs=pd.concat(dfs)

In [38]:
df=dfs.join(tickers.set_index('symbol')['name'],on='ticker')
df['name']=df['name'].str.replace(' INC','').str.title()

df['diff']=df['close']-df['open']
direction=[]
for i in df['diff'].values:
    if i<0:
        direction.append('minus')
    else:
        direction.append('plus')
df['direction']=direction
df['diff']=abs(df['diff'])

direction=[]
for i in df['MACDh_12_26_9'].values:
    if i<0:
        direction.append('minus')
    else:
        direction.append('plus')
df['MACDh_direction']=direction

df['bollingerm']=df['close']-df['BBM_20_2.0']
df['bollingeru']=df['close']-df['BBU_20_2.0']
df['bollingerl']=df['close']-df['BBL_20_2.0']
direction=[]
for i in df[['bollingerm','bollingeru','bollingerl']].values:
    if i[0]>0.1:
        if i[1]>0:
            direction.append(-1)
        else:
            direction.append(-0.5)
    elif i[0]<-0.1:
        if i[2]>0:
            direction.append(0.5)
        else:
            direction.append(1)
    else:
        direction.append(0)
df['bollinger']=direction

In [39]:
df1=df[df['ticker'].isin(['AAPL','AAL'])]

In [40]:
tag_columns=['name','direction','ticker','MACDh_direction']
field_columns=[i for i in df.columns if i not in tag_columns]
measurement='monthly'
push2influx(df1,measurement,field_columns,tag_columns,fo=True)

Purging monthly ...
Writing to monthly ...
84 data points will be written in 0.0168 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!
